https://es.wallapop.com/search?source=search_box&category_id=24200&keywords=telefonia

https://es.wallapop.com/search?source=search_box&category_id=24200&keywords=impresoras+3d

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import csv
import time
import random

# ⚙️ Configurar navegador (modo visible recomendado para depurar)
options = Options()
# options.add_argument("--headless")  # Puedes quitarlo para ver el navegador
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(options=options)
url = "https://es.wallapop.com/search?category_id=24200&subcategory_ids=24202&source=side_bar_filters"
driver.get(url)
time.sleep(10)  # Espera inicial larga para carga completa

# 🔁 Scroll dinámico hasta que no cargue más productos
prev_count = -1
scrolls = 0

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(random.uniform(2.5, 4))  # Espera aleatoria

    # Rebote para activar carga dinámica
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight - 400);")
    time.sleep(1)

    productos = driver.find_elements(By.CSS_SELECTOR, "a.item-card_ItemCard--vertical__FiFz6")
    current_count = len(productos)
    print(f"🔄 Scroll #{scrolls + 1}: {current_count} productos")

    if current_count == prev_count:
        print("✅ No se cargan más productos. Fin del scroll.")
        break

    prev_count = current_count
    scrolls += 1

# 📦 Guardar datos en CSV
print(f"\n📦 Total de productos extraídos: {len(productos)}")

with open("productos_wallapop.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Título", "Precio", "URL", "Imagen"])

    urls_vistas = set()  # Para evitar duplicados

    for producto in productos:
        try:
            url = producto.get_attribute("href")
            if url in urls_vistas:
                continue
            urls_vistas.add(url)

            titulo = producto.find_element(By.CSS_SELECTOR, "h3.item-card_ItemCard__title__8eq2b").text.strip()
            precio = producto.find_element(By.CSS_SELECTOR, "strong.item-card_ItemCard__price__D3QWU").text.strip()
            imagen = producto.find_element(By.CSS_SELECTOR, "img").get_attribute("src")

            print(f"✅ {titulo} | {precio}")
            writer.writerow([titulo, precio, url, imagen])
        except Exception as e:
            print("⚠️ Producto omitido:", e)

driver.quit()
print("\n✅ CSV generado: productos_wallapop.csv")